In [2]:

# importando bibliotecas essenciais para a realização do programa

from collections import defaultdict, deque # defaultdict para simular a lista de adjacência, deque para simular pilha
import requests # requests para requisição da base de dados hospedada no github
import csv # csv para leitura da base de dados
from io import StringIO # stringIO para criar um objeto a partir do conteudo
import matplotlib.pyplot as plt # plotagem de graficos
import math
import heapq

url = 'https://raw.githubusercontent.com/Fosswt/TDE5_GRAPHS/main/netflix_amazon_disney_titles.csv'

### Grafo Ponderado Direcionado

In [ ]:

class grafo_direcionado:
    def __init__(self):
        self.lista_adjacencia = defaultdict(lambda: defaultdict(int))  # lista de adjacência com pesos
        self.vertices = set()  # conjunto para armazenar os vértices do grafo
        self.ordem = 0  # número de vértices no grafo
        self.tamanho = 0  # número de arestas no grafo
        self.grau_total = defaultdict(int)
        self.construir_grafo()

    def adicionar_aresta(self, ator, diretor):
        ator = ator.upper()
        diretor = diretor.upper()
        if diretor not in self.lista_adjacencia[ator]:
            self.lista_adjacencia[ator][diretor] = 0  # inicializa a aresta com 0 se não existe
        self.lista_adjacencia[ator][diretor] += 1  # incrementa o peso da aresta
        self.vertices.add(ator)
        self.vertices.add(diretor)
        self.grau_total[ator] += 1  # Incrementar o grau de saída do ator
        self.grau_total[diretor] += 1  # Incrementar o grau de entrada do diretor
        self.tamanho += 1  # incrementa o número de arestas
        self.ordem = len(self.vertices)

    def construir_grafo(self):
        # a base de dados foi hospedada no github afim de facilitar a execução do código, sem precisar importar o arquivo csv
        # toda vez que for rodar.
        response = requests.get(url) # fazendo o request para obter o conteúdo do arquivo
        if response.status_code == 200: # verificar se a requisição foi realizada com codigo 200 (ok)
            content = response.content.decode('utf-8') # decodificando o conteudo de bytes para texto
            csv_file = StringIO(content) # criando um objeto StringIO a partir do conteudo
            reader = csv.DictReader(csv_file) # usando csv.DictReader para ler o arquivo .csv
            for row in reader:
                if not row['cast'] or not row['director']:
                    continue  # ignora linhas com valores vazios
                # limpa e divide os valores das colunas 'cast' e 'director'
                diretores = [d.strip() for d in row['director'].split(',') if d.strip()] # removendo espaços em branco
                atores = [a.strip() for a in row['cast'].split(',') if a.strip()]
                if not diretores or not atores: # adiciona arestas somente se houver valores válidos
                    continue
                for diretor in diretores:
                    for ator in atores:
                        self.adicionar_aresta(ator, diretor)
        self.ordem = len(self.vertices)  # atualiza o número de vértices

    def mostrar_lista_adjacencia(self):
        for ator, diretores in self.lista_adjacencia.items():
            diretores_str = ', '.join(f'{diretor} (Colaborações: {peso})' for diretor, peso in diretores.items())
            print(f'{ator} => {diretores_str}')

    def contar_vertices(self):
        return len(self.vertices)

    def contar_arestas(self):
        return self.tamanho

    def dfs(self, v, visitado, pilha):
        visitado[v] = True
        for vizinho in self.lista_adjacencia[v]:
            if not visitado[vizinho]:
                self.dfs(vizinho, visitado, pilha)
        pilha.append(v)

    def dfs_inverso(self, v, visitado, componente):
        # Marca o vértice atual como visitado
        visitado[v] = True
        # Adiciona o vértice atual à lista do componente
        componente.append(v)
        for vizinho in self.lista_adjacencia[v].keys():
          # Se o vizinho não foi visitado ainda, chama recursivamente o dfs_inverso para o vizinho
            if not visitado[vizinho]:
                self.dfs_inverso(vizinho, visitado, componente)

    # 2) Função para a identificação de componentes.
    def encontrar_componentes_conectados(self): # encontrando componentes fortemente conectados utilizando algoritmo de kosaraju
        visitado = {v: False for v in self.vertices}
        pilha_ordem_topologica = []

        # primeira busca em profundidade para preencher a pilha de ordem topológica
        for v in self.vertices:
            if not visitado[v]:
                self.dfs(v, visitado, pilha_ordem_topologica)

        # grafo transposto (inverte todas as arestas)
        grafo_transposto = defaultdict(lambda: defaultdict(int))
        for v in self.vertices:
            for vizinho, peso in self.lista_adjacencia[v].items():
                grafo_transposto[vizinho][v] = peso

        # reiniciar o vetor visitado para a segunda busca em profundidade
        visitado = {v: False for v in self.vertices}
        componentes_fortemente_conectados = []

        # segunda DFS na ordem inversa da pilha de ordem topológica
        while pilha_ordem_topologica:
            v = pilha_ordem_topologica.pop()
            if not visitado[v]:
                componente = []
                self.dfs_inverso(v, visitado, componente)
                componentes_fortemente_conectados.append(componente)

        return componentes_fortemente_conectados

    # 3) Função que recebe como entrada um vértice X (por exemplo, BOB ODENKIRK) e retorna a Árvore Geradora Mínima da componente que contêm o vértice X
    def mst(self, vertice):
        vertice = vertice.upper()
        if vertice not in self.vertices:
            raise ValueError(f"Vértice {vertice} não está presente no grafo.")

        # Encontrar a componente fortemente conectada que contém o vértice
        componentes = self.encontrar_componentes_conectados()
        componente_x = None

        # Procura a componente que contém o vértice fornecido
        for componente in componentes:
            if vertice in componente:
                componente_x = componente
                break

        if componente_x is None:
            raise ValueError(f"Vértice {vertice} não está em nenhuma componente fortemente conectada.")

        # Criar um novo grafo com apenas os vértices da componente
        grafo_componente = self
        for v in componente_x:
            grafo_componente.vertices.add(v) # Adiciona os vértices da componente ao novo grafo
            for u, peso in self.lista_adjacencia[v].items():
                if u in componente_x:
                    grafo_componente.lista_adjacencia[v][u] = peso # Adiciona as arestas que estão dentro da componente

        # Aplicar o algoritmo de Prim para encontrar a Árvore Geradora Mínima
        arvore_geradora = []
        visitados = set()
        heap = [(0, vertice)]
        custo_total = 0

        # Verifica se há elementos ainda
        while heap:
            # Remove e retorna a aresta de menor peso do heap
            peso, u = heapq.heappop(heap)
            # Se o vértice não foi visitado ainda
            if u not in visitados:
                visitados.add(u) # Marca o vértice como visitado
                custo_total += peso # Adiciona o peso da aresta ao custo total
                arvore_geradora.append((u, peso)) # Adiciona a aresta à MST
                # Itera sobre os vizinhos do vértice u
                for v, peso in grafo_componente.lista_adjacencia[u].items():
                    if v not in visitados:
                        heapq.heappush(heap, (peso, v)) # Adiciona a aresta ao heap

        return arvore_geradora, custo_total


    # 4) Função que calcula a Centralidade de Grau (Degree Centrality) de um vértice.
    def centralidade_grau(self, vertice):
        vertice = vertice.upper()
        if vertice not in self.grau_total or self.ordem <= 1:
            return 0.0
        # Normalização pelo número de vértices
        return self.grau_total[vertice] / (self.ordem - 1)

    def top10_centrais(self):
        # Obter todos os vértices e suas centralidades
        centralidades = [(vertice, self.centralidade_grau(vertice)) for vertice in self.vertices]

        # Ordenar por centralidade (grau total) em ordem decrescente
        centralidades.sort(key=lambda x: x[1], reverse=True)

        # Retornar os 10 vértices com maior centralidade
        return centralidades[:10]

    # 5) função que cria grafico de barras pra mostrar top 10 vertices com maiores valores de centralidade de grau
    def plotar_grafico_centralidade(self):
        top10 = self.top10_centrais()
        vertices = [v[0] for v in top10]
        centralidades = [v[1] for v in top10]

        plt.figure(figsize=(6, 3))
        plt.bar(vertices, centralidades, color='cyan')
        plt.xlabel('Vértices')
        plt.ylabel('Centralidade de Grau')
        plt.title('Top 10 Vértices com Maior Centralidade de Grau')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()

    # 6) Função que calcula a Centralidade de Intermediação (Betweenness Centrality) de um vértice.
    def calcular_centralidade_intermediacao_ponderada(self):
        centralidade = defaultdict(float)

        for s in self.vertices:
            # Inicialização
            pilha = []  # Pilha para armazenar a ordem dos vértices processados
            predecessors = defaultdict(list)  # Dicionário para armazenar os predecessores dos vértices
            contagem_caminhos_curtos = defaultdict(int)  # Contagem de caminhos mais curtos
            distancia = defaultdict(lambda: float('inf'))  # Distâncias dos vértices a partir da origem
            contagem_caminhos_curtos[s] = 1  # Número de caminhos mais curtos do vértice s para ele mesmo é 1
            distancia[s] = 0  # Distância do vértice s para ele mesmo é 0

            # Algoritmo de Dijkstra
            fila_prioridade = [(0, s)]
            while fila_prioridade:
                 # Remove e obtém o vértice com a menor distância conhecida
                dist_v, v = heapq.heappop(fila_prioridade)
                # Se a distância do vértice v já foi atualizada, continua
                if dist_v > distancia[v]:
                    continue
                pilha.append(v)

                # Itera sobre cada vizinho w de v e o peso da aresta (v, w)
                for w, peso in self.lista_adjacencia[v].items():
                    vw_dist = dist_v + peso # Calcula a nova distância proposta para w
                    # Se a nova distância para w é menor que a registrada
                    if vw_dist < distancia[w]:
                        distancia[w] = vw_dist # Atualiza a distância de w
                        heapq.heappush(fila_prioridade, (vw_dist, w)) # Adiciona w à fila de prioridade
                        contagem_caminhos_curtos[w] = contagem_caminhos_curtos[v] # Atualiza a contagem de caminhos curtos para w
                        predecessors[w] = [v] # Define v como predecessor de w
                    elif vw_dist == distancia[w]:
                        contagem_caminhos_curtos[w] += contagem_caminhos_curtos[v] # Atualiza a contagem de caminhos curtos para w
                        predecessors[w].append(v) # Adiciona v como predecessor de w

            # Acumulação
            delta = defaultdict(float) # Acumula centralidade
            while pilha:
                w = pilha.pop()
                for v in predecessors[w]:
                    delta[v] += (contagem_caminhos_curtos[v] / contagem_caminhos_curtos[w]) * (1 + delta[w])
                if w != s:
                    centralidade[w] += delta[w]

        # Normalização
        for v in centralidade:
            centralidade[v] /= 2

        return centralidade

    # Pegar apenas os 10 maiores nós com maior centralidade de intermediação
    def calcular_centralidade_intermediacao_para_x(self, x):
        centralidade = self.calcular_centralidade_intermediacao_ponderada()
        return centralidade[x.upper()]

    # Pegar apenas os 10 nós com maiores valores de centralidade de intermediacao
    def calcular_top_10_centralidade_intermediacao(self):
        centralidade = self.calcular_centralidade_intermediacao_ponderada()
        top_10 = heapq.nlargest(10, centralidade.items(), key=lambda x: x[1])
        return top_10

    # Plotar grafico dos 10 nós com mais centralidade de intermediacao
    def plotar_grafico_intermediacao(self):
        top10 = self.calcular_top_10_centralidade_intermediacao()
        vertices = [v[0] for v in top10]
        proximidade = [v[1] for v in top10]

        plt.figure(figsize=(6, 3))
        plt.bar(vertices, proximidade, color='purple')
        plt.xlabel('Vértices')
        plt.ylabel('Centralidade de Intermediação')
        plt.title('Top 10 Vértices com Maior Centralidade de Intermediação')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()

    def dijkstra_distancia(self, inicio):
        distancia = defaultdict(lambda: float('inf'))
        distancia[inicio] = 0
        fila_prioridade = [(0, inicio)]

        while fila_prioridade:
            dist_u, u = heapq.heappop(fila_prioridade)
            if dist_u > distancia[u]:
                continue
            for v, peso in self.lista_adjacencia[u].items():
                alt = dist_u + peso
                if alt < distancia[v]:
                    distancia[v] = alt
                    heapq.heappush(fila_prioridade, (alt, v))

        return distancia

    def centralidade_proximidade(self):
        proximidade = {}
        num_vertices = len(self.lista_adjacencia)

        for v in self.lista_adjacencia:
            distancias = self.dijkstra_distancia(v)
            # Calcula a soma das distâncias de x para todos os outros vértices, ignorando distâncias infinitas
            distancia_total = sum(distancias[w] for w in distancias if distancias[w] < float('inf'))
            if distancia_total > 0:
                proximidade[v] = (num_vertices - 1) / distancia_total
            else:
                proximidade[v] = 0.0

        return proximidade

    # Calcular a centralidade de proximidade para um vértice específico
    def centralidade_proximidade_para_x(self, x):
        x = x.upper()
        # Verifica se o vértice x está presente no grafo
        if x not in self.vertices:
            raise ValueError(f"Vértice {x} não está presente no grafo.")

        # Calcula as distâncias de x para todos os outros vértices usando o algoritmo de Dijkstra
        distancias = self.dijkstra_distancia(x)
        # Obtém o número total de vértices no grafo
        num_vertices = len(self.lista_adjacencia)
        # Calcula a soma das distâncias de x para todos os outros vértices, ignorando distâncias infinitas
        distancia_total = sum(distancias[w] for w in distancias if distancias[w] < float('inf'))
        # Verifica se a distância total é maior que 0 para evitar divisão por zero
        if distancia_total > 0:
            return (num_vertices - 1) / distancia_total
        else:
            # Evitar divisão por 0
            return 0.0


    # Pegar apenas os 10 nós com maiores valores de centralidade de proximidade
    def top_10_centralidade_proximidade(self):
        proximidade = self.centralidade_proximidade()
        top_10 = heapq.nlargest(10, proximidade.items(), key=lambda x: x[1])
        return top_10

    # Plotar gráfico de barras com os 10 nós com maiores valores de centralidade de proximidade
    def plotar_grafico_proximidade(self):
        top10 = self.top_10_centralidade_proximidade()
        vertices = [v[0] for v in top10]
        proximidade = [v[1] for v in top10]

        plt.figure(figsize=(6, 3))
        plt.bar(vertices, proximidade, color='purple')
        plt.xlabel('Vértices')
        plt.ylabel('Centralidade de Intermediação')
        plt.title('Top 10 Vértices com Maior Centralidade de Proximidade')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()




#funções principais do grafo direcionado
grafo_direcionado = grafo_direcionado()



print(f'Número de vértices: {grafo_direcionado.contar_vertices()}')
print(f'Número de arestas: {grafo_direcionado.contar_arestas()}')
print(f'Número de componentes fortemente conectadas (kosaraju): {len(grafo_direcionado.encontrar_componentes_conectados())}')

arvore_geradora, custo_total = grafo_direcionado.mst('bob odenkirk')
print("Árvore Geradora Mínima:", arvore_geradora)
print("Custo Total:", custo_total)


print('top 10 vertices com maiores valores de centralidade de grau: ')
grafo_direcionado.plotar_grafico_centralidade()

print('top 10 vertices com maiores valores de centralidade de intermediação:')
print(grafo_direcionado.plotar_grafico_intermediacao())

print('top 10 vertices com maiores valores de centralidade de proximidade:') # valores do top 10 são completamente iguais.
# quando chamamos o centralidade_proximidade_para_x() para um vértice, ele retorna um valor ok
print(grafo_direcionado.top_10_centralidade_proximidade())
grafo_direcionado.plotar_grafico_proximidade()
#print(grafo_direcionado.centralidade_proximidade_para_x('bob odenkirk'))


### Grafo Ponderado Não-Direcionado

In [ ]:

class grafo_nao_direcionado:
    def __init__(self):
        self.lista_adjacencia = defaultdict(lambda: defaultdict(int))  # lista de adjacência com pesos
        self.vertices = set()  # conjunto para armazenar os vértices do grafo
        self.ordem = 0  # numero de vértices no grafo
        self.tamanho = 0  # numero de arestas no grafo
        self.construir_grafo()

    def contar_vertices(self):
        return len(self.vertices)

    def contar_arestas(self):
        return self.tamanho

    def adicionar_aresta(self, ator1, ator2):
        ator1 = ator1.upper()
        ator2 = ator2.upper()
        if ator1 != ator2:  # evita conexões de um ator consigo mesmo
            self.lista_adjacencia[ator1][ator2] += 1
            self.lista_adjacencia[ator2][ator1] += 1  # adiciona a aresta na direção oposta
        else:
            self.lista_adjacencia[ator1][ator2] += 1  # laços são adicionados uma única vez

        self.vertices.add(ator1)
        self.vertices.add(ator2)
        self.tamanho += 1  #incrementa o número de arestas

    def construir_grafo(self):
        response = requests.get(url)
        if response.status_code == 200:
            content = response.content.decode('utf-8')
            csv_file = StringIO(content)
            reader = csv.DictReader(csv_file)
            for row in reader:
                if not row['cast']:
                    continue  # Ignora linhas com valores vazios na coluna 'CAST'

                # Limpa, divide e converte para caixa alta os valores da coluna 'CAST'
                atores = [a.strip().upper() for a in row['cast'].split(',') if a.strip()]

                # Adiciona arestas entre todos os pares de atores
                for i in range(len(atores)):
                    for j in range(i + 1, len(atores)):
                        self.adicionar_aresta(atores[i], atores[j])

        self.ordem = len(self.vertices)  # Atualiza o número de vértices

    # Printar lista de adjacencias
    def mostrar_lista_adjacencia(self):
        for ator, conexoes in self.lista_adjacencia.items():
            conexoes_str = ', '.join(f'{conectado} (Peso: {peso})' for conectado, peso in conexoes.items())
            print(f'{ator} -> {conexoes_str}')

    # Busca em profundidade
    def dfs(self, v, visitado):
            pilha = [v]
            while pilha:
                atual = pilha.pop()
                if not visitado[atual]:
                    visitado[atual] = True
                    for vizinho in self.lista_adjacencia[atual]:
                        if not visitado[vizinho]:
                            pilha.append(vizinho)

    def encontrar_componentes_conectados(self):
        # Inicializa um dicionário para marcar todos os vértices como não visitados
        visitado = {v: False for v in self.vertices}

        # Inicializa o contador de componentes conectados
        num_componentes = 0

        # Itera sobre cada vértice no grafo
        for v in self.vertices:
            # Se o vértice não foi visitado ainda
            if not visitado[v]:
                # Realiza uma busca em profundidade (DFS) a partir desse vértice
                self.dfs(v, visitado)
                # Incrementa o contador de componentes conectados
                num_componentes += 1

        # Retorna o número total de componentes conectados no grafo
        return num_componentes

    def prim_mst(self, vertice_inicial):
        vertice_inicial = vertice_inicial.upper()

        # Verifica se o vértice inicial pertence ao grafo
        if vertice_inicial not in self.vertices:
            return [], 0  # Se não pertence, retorna uma árvore mínima vazia e custo total 0

        mst = []  # Lista para armazenar as arestas da árvore geradora mínima (MST)
        visitados = set()  # Conjunto para manter os vértices visitados
        custo_total = 0  # Variável para somar o custo total da MST

        heap = [(0, vertice_inicial)]  # Inicializa o heap com o vértice inicial e peso 0

        while heap and len(mst) < self.ordem - 1:
            peso, u = heapq.heappop(heap)  # Remove e retorna a aresta de menor peso do heap
            if u in visitados:
                continue  # Se o vértice já foi visitado, continua para a próxima iteração
            visitados.add(u)  # Marca o vértice como visitado
            custo_total += peso  # Adiciona o peso da aresta ao custo total

            # Itera sobre os vizinhos do vértice u
            for v, peso_uv in self.lista_adjacencia[u].items():
                if v not in visitados:  # Se o vizinho não foi visitado
                    heapq.heappush(heap, (peso_uv, v))  # Adiciona a aresta ao heap
                    mst.append((u, v, peso_uv))  # Adiciona a aresta à MST

        return mst, custo_total  # Retorna a lista de arestas da MST e o custo total

    def centralidade_grau(self, vertice):
        vertice = vertice.upper()
        grau_total = sum(self.lista_adjacencia[vertice].values())
        # Normalização pelo número de vértices
        return grau_total / (self.ordem - 1)

    def top10_centrais(self):
        # Obter todos os vértices e suas centralidades
        centralidades = [(vertice, self.centralidade_grau(vertice)) for vertice in self.vertices]

        # Ordenar por centralidade (grau total) em ordem decrescente
        centralidades.sort(key=lambda x: x[1], reverse=True)

        # Retornar os 10 vértices com maior centralidade
        return centralidades[:10]

    def plotar_grafico_centralidade(self):
        top10 = self.top10_centrais()
        vertices = [v[0] for v in top10]
        centralidades = [v[1] for v in top10]

        plt.figure(figsize=(6, 3))
        plt.bar(vertices, centralidades, color='cyan')
        plt.xlabel('Vértices')
        plt.ylabel('Centralidade de Grau')
        plt.title('Top 10 Vértices com Maior Centralidade de Grau')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()

    def dijkstra_distancia(self, inicio):
        distancia = defaultdict(lambda: float('inf'))
        distancia[inicio] = 0
        fila_prioridade = [(0, inicio)]

        while fila_prioridade:
            dist_u, u = heapq.heappop(fila_prioridade)
            if dist_u > distancia[u]:
                continue
            for v, peso in self.lista_adjacencia[u].items():
                alt = dist_u + peso
                if alt < distancia[v]:
                    distancia[v] = alt
                    heapq.heappush(fila_prioridade, (alt, v))

        return distancia

    def centralidade_proximidade(self):
        proximidade = {}
        num_vertices = len(self.lista_adjacencia)

        for v in self.lista_adjacencia:
            # Calcula as distâncias mínimas do vértice 'v' para todos os outros vértices usando Dijkstra
            distancias = self.dijkstra_distancia(v)
            # Calcula a soma das distâncias mínimas do vértice 'v' para todos os outros vértices acessíveis
            distancia_total = sum(distancias[w] for w in distancias if distancias[w] < float('inf'))
            # Se a soma das distâncias é maior que 0, calcula a centralidade de proximidade
            if distancia_total > 0:
                proximidade[v] = (num_vertices - 1) / distancia_total
            else:
                # Caso contrário, define a centralidade de proximidade como 0
                proximidade[v] = 0.0

        return proximidade

    def centralidade_proximidade_para_x(self, x):
        x = x.upper()
        if x not in self.vertices:
            raise ValueError(f"Vértice {x} não está presente no grafo.")

        distancias = self.dijkstra_distancia(x)
        num_vertices = len(self.lista_adjacencia)
        distancia_total = sum(distancias[w] for w in distancias if distancias[w] < float('inf'))
        if distancia_total > 0:
            return (num_vertices - 1) / distancia_total
        else:
            return 0.0

    def top_10_centralidade_proximidade(self):
        proximidade = self.centralidade_proximidade()
        top_10 = heapq.nlargest(10, proximidade.items(), key=lambda x: x[1])
        return top_10

    def calcular_centralidade_intermediacao_ponderada(self):
        centralidade = defaultdict(float)

        for s in self.vertices:
            # Inicialização
            pilha = []
            predecessors = defaultdict(list)
            contagem_caminhos_curtos = defaultdict(int)
            distancia = defaultdict(lambda: float('inf'))
            contagem_caminhos_curtos[s] = 1
            distancia[s] = 0

            # Algoritmo de Dijkstra modificado para um grafo não direcionado
            fila_prioridade = [(0, s)]
            while fila_prioridade:
                dist_v, v = heapq.heappop(fila_prioridade)
                if dist_v > distancia[v]:
                    continue
                pilha.append(v)
                for w, peso in self.lista_adjacencia[v].items():
                    vw_dist = dist_v + peso
                    if vw_dist < distancia[w]:
                        distancia[w] = vw_dist
                        heapq.heappush(fila_prioridade, (vw_dist, w))
                        contagem_caminhos_curtos[w] = contagem_caminhos_curtos[v]
                        predecessors[w] = [v]
                    elif vw_dist == distancia[w]:
                        contagem_caminhos_curtos[w] += contagem_caminhos_curtos[v]
                        predecessors[w].append(v)

            # Acumulação de centralidade de intermediação ponderada
            delta = defaultdict(float)
            while pilha:
                w = pilha.pop()
                for v in predecessors[w]:
                    delta[v] += (contagem_caminhos_curtos[v] / contagem_caminhos_curtos[w]) * (1 + delta[w])
                if w != s:
                    centralidade[w] += delta[w]

        # Normalização
        num_vertices = len(self.vertices)
        for v in centralidade:
            centralidade[v] /= ((num_vertices - 1) * (num_vertices - 2)) / 2  # Normalização para grafos não direcionados

        return centralidade

    def centralidade_intermediacao_para_vertice(self, vertice):
        centralidade = self.calcular_centralidade_intermediacao_ponderada()
        return centralidade[vertice.upper()]

    def top_10_centralidade_intermediacao(self):
        centralidade = self.calcular_centralidade_intermediacao_ponderada()
        top_10 = heapq.nlargest(10, centralidade.items(), key=lambda x: x[1])
        return top_10

    def plotar_grafico_intermediacao(self):
        top10 = self.top_10_centralidade_intermediacao()
        vertices = [v[0] for v in top10]
        intermediacao = [v[1] for v in top10]

        plt.figure(figsize=(8, 4))
        plt.bar(vertices, intermediacao, color='purple')
        plt.xlabel('Vértices')
        plt.ylabel('Centralidade de Intermediação')
        plt.title('Top 10 Vértices com Maior Centralidade de Intermediação (Grafo Não Direcionado)')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()

    def calcular_centralidade_proximidade(self, vertice):
        vertice = vertice.upper()
        if vertice not in self.vertices:
            return 0.0

        # Algoritmo de Dijkstra para encontrar caminhos mais curtos
        fila_prioridade = [(0, vertice)]
        visitados = set()
        distancias = {v: float('inf') for v in self.vertices}
        distancias[vertice] = 0

        while fila_prioridade:
            dist, v = heapq.heappop(fila_prioridade)
            if v in visitados:
                continue
            visitados.add(v)
            for vizinho, peso in self.lista_adjacencia[v].items():
                nova_distancia = dist + peso
                if nova_distancia < distancias[vizinho]:
                    distancias[vizinho] = nova_distancia
                    heapq.heappush(fila_prioridade, (nova_distancia, vizinho))

        # Calcular a centralidade de proximidade
        soma_distancias = sum(distancias[v] for v in self.vertices if v != vertice)
        if soma_distancias == 0:
            return 0.0
        centralidade = 1 / soma_distancias
        return centralidade

    def calcular_top_10_centralidade_proximidade(self):
        centralidades = [(v, self.calcular_centralidade_proximidade(v)) for v in self.vertices]
        top_10 = sorted(centralidades, key=lambda x: x[1], reverse=True)[:10]
        return top_10

    def plotar_grafico_proximidade(self):
        top10 = self.calcular_top_10_centralidade_proximidade()
        vertices = [v[0] for v in top10]
        proximidades = [v[1] for v in top10]

        plt.figure(figsize=(8, 4))
        plt.bar(vertices, proximidades, color='green')
        plt.xlabel('Vértices')
        plt.ylabel('Centralidade de Proximidade')
        plt.title('Top 10 Vértices com Maior Centralidade de Proximidade')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()

# Exemplo de uso
grafo_nao_direcionado = grafo_nao_direcionado()
print(f'Número de vértices: {grafo_nao_direcionado.contar_vertices()}')
print(f'Número de arestas: {grafo_nao_direcionado.contar_arestas()}')
print(f'Número de componentes conectados: {grafo_nao_direcionado.encontrar_componentes_conectados()}')


vertice = 'bob odenkirk'
mst, custo_total = grafo_nao_direcionado.prim_mst(vertice)

# Mostrando a MST e o custo total
print(f"Árvore Geradora Mínima (MST) para o vértice <{mst}>:")
print(f"Custo Total da Árvore: {custo_total}")

print('top 10 vertices com maiores valores de centralidade de grau: ')
print(grafo_nao_direcionado.top10_centrais())
grafo_nao_direcionado.plotar_grafico_centralidade()

print('top 10 vertices com maiores valores de centralidade de intermediação: ')
print(grafo_nao_direcionado.top_10_centralidade_intermediacao())
print(grafo_nao_direcionado.plotar_grafico_intermediacao())

print('top 10 vertices com maiores valores de centralidade de proximidade: ')
print(grafo_nao_direcionado.top_10_centralidade_proximidade())
print(grafo_nao_direcionado.plotar_grafico_proximidade())






# grafo_nao_direcionado.mostrar_lista_adjacencia()


